In [49]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
import json
from ollama import Client
from IPython.display import display_markdown, display_latex

from src.tool import ToolKit, ToolCallProcessor
from functions import summation, multiplication

### Connect to local Ollama_server

In [51]:
client = Client(host="http://localhost:11434")

model_name = "qwen2.5:7b"

# Pull a model
result = client.pull(model=model_name)
result

{'status': 'success'}

### Registry functions

In [52]:
# Instanciar o registro de ferramentas
toolkit = ToolKit(tools=[summation, multiplication])

### Build the prompt

In [53]:
tools = "\n\n".join([json.dumps(schema, indent=4) for schema in toolkit.tools_schemas()])


TOOL_PROMPT = """
You are a function calling AI model.
If a function or tool is unavailable, responde with trained data.
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags, here are the available tools:
<tools>
%s
</tools>

For each function call, to a listed function, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>, "id": <monotonically-increasing-id>}
</tool_call>
""" % tools

In [54]:
tool_chat_history = [
    {
        "role": "system",
        "content": TOOL_PROMPT,
    }
]

agent_chat_history = []

USER_PROMPT = "Can you multiple 25 by 12 and sum 3 with 6 and search what is an LLM on google.com?"

tool_chat_history.append(
    {"role": "user", "content": USER_PROMPT}
)
agent_chat_history.append(
    {"role": "user", "content": USER_PROMPT}
)

# Request tools action
tools_calls_request = client.chat(model=model_name, messages=tool_chat_history)
# Parse the response into model default format
tools_calls_str = tools_calls_request['message']['content']

tool_calls = ToolCallProcessor(tools_calls_str)

In [55]:
# Return the final response using the tools retrivied info
for call in tool_calls.tool_calls:
    try:
        result = toolkit.get_tool_by_name(call.name).run(call.arguments)

        agent_chat_history.append({
            "role": "tool",
            "content": f"result of call to {call.name}: {result}"
        })
    except AttributeError as ae:
        print(ae)

In [56]:
tool_response = client.chat(model=model_name, messages=agent_chat_history)

response_using_tool = tool_response['message']['content']

display_markdown(response_using_tool, raw=True)

Sure, the result of multiplying 25 by 12 is 300, and the sum of 3 and 6 is 9.

Now, regarding your request to search for what an LLM is on Google, while I can't directly perform a web search, I can tell you that LLM stands for Large Language Model. These are AI models designed to understand and generate human-like text. They're used in various applications like chatbots, content generation, and more. If you need more detailed information, I can help guide you on how to find it yourself or provide some general details about LLMs.